In [72]:
import json
import re
from typing import NamedTuple

import pandas as pd

laptops = pd.read_csv('data/canada_computers_laptops_gpt.csv')

# Assign "gpu" column based on specs['GPU']
laptops['specs'] = laptops['specs'].apply(lambda x: eval(x))
laptops['gpt_specs'] = laptops['gpt_specs'].apply(lambda x: eval(x))
laptops['GPU'] = laptops['gpt_specs'].apply(lambda x: x['GPU'] if 'GPU' in x else None)
laptops['CPU'] = laptops['gpt_specs'].apply(lambda x: x['CPU'] if 'CPU' in x else None)
laptops.head(10)

,id,item_id,name,brand,categories,price,specs,gpt_specs,GPU,CPU
0,234878,LPASU00715,"ASUS TUF F15 Gaming Laptop 15.6"" FHD Intel i9-...",ASUS,"['Laptops, Tablet', None, None, None]",1499.0,"{'Refresh Rate': '144Hz', 'GPU': 'GeForce RTX ...","{'Brand': 'ASUS', 'Model': 'TUF F15', 'Model C...",GeForce RTX 4060,Intel i9-13900H
1,239821,LPACE00967,"Acer Nitro 5 Gaming Laptop 15.6"" FHD Intel i9-...",ACER,"['Laptops, Tablet', None, None, None]",1299.0,"{'GPU': 'GeForce RTX 4060', 'Maximum Resolutio...","{'Brand': 'Acer', 'Model': 'Nitro 5', 'Model C...",GeForce RTX 4060,Intel i9-12900H
2,241721,LPLEN03469,"Lenovo Legion Pro 5i Gaming Laptop 16"" QHD+ In...",LENOVO,"['Laptops, Tablet', None, None, None]",1499.0,"{'GPU': 'GeForce RTX 4060', 'Maximum Resolutio...","{'Brand': 'Lenovo', 'Model': 'Legion Pro 5i', ...",GeForce RTX 4060,Intel i7-13700HX
3,245321,LPHEW02100,"HP 15-fd0030ca Consumer Laptop 15.6"" FHD Intel...",HP,"['Laptops, Tablet', None, None, None]",699.0,"{'GPU': 'Intel Iris Xe', 'Maximum Resolution':...","{'Brand': 'HP', 'Model': '15-fd0030ca', 'Model...",None,Intel i5-1340P
4,245308,LPGIG00162,"GIGABYTE G5 KF Gaming Laptop 15.6"" FHD 144Hz I...",GIGABYTE,"['Laptops, Tablet', None, None, None]",1099.0,"{'GPU': 'GeForce RTX 4060', 'Maximum Resolutio...","{'Brand': 'GIGABYTE', 'Model': 'G5 KF', 'Model...",GeForce RTX 4060,Intel i7-12650H
5,230569,LPHEW01946,"HP ProBook 450 G9 15.6"" Business Notebook Inte...",HP,"['Laptops, Tablet', None, None, None]",899.0,{'General Information<HDRBR>Manufacturer': 'HP...,"{'Brand': 'HP', 'Model': 'ProBook 450 G9', 'CP...",None,Intel i5-1235U
6,241145,LPMSI00572,"MSI Thin GF63 Gaming Laptop 15.6"" FHD Intel i5...",MSI,"['Laptops, Tablet', None, None, None]",699.0,"{'GPU': 'GeForce RTX 2050', 'Maximum Resolutio...","{'Brand': 'MSI', 'Model': 'Thin GF63', 'CPU': ...",GeForce RTX 2050,Intel i5-12450H
7,243411,LPLEN03502,"LENOVO IdeaPad Slim 3 Consumer Laptop 15.6"" AM...",LENOVO,"['Laptops, Tablet', None, None, None]",699.0,"{'GPU': 'AMD Radeon 610M', 'Maximum Resolution...","{'Brand': 'LENOVO', 'Model': 'IdeaPad Slim 3',...",None,AMD Ryzen 5 7520U
8,243633,LPACE00980,"ACER Nitro 5 Gaming Laptop 15.6"" 144Hz AMD Ryz...",ACER,"['Laptops, Tablet', None, None, None]",999.0,"{'GPU': 'GeForce RTX 3060', 'Maximum Resolutio...","{'Brand': 'ACER', 'Model': 'Nitro 5', 'CPU': '...",GeForce RTX 3060,AMD Ryzen 7 5800H
9,240323,LPHEW02049,"HP EliteBook 840 G2 Business Notebook 14"" Touc...",HP,"['Laptops, Tablet', None, None, None]",269.0,"{'Memory': '16GB', 'Storage Type': 'SSD', 'CPU...","{'Brand': 'HP', 'Model': 'EliteBook 840 G2', '...",None,Core i5-5300U


In [73]:
import csv

# cpu_records = list(csv.DictReader(open('data/notebookcheck/mobile-cpu.csv', 'r', encoding='utf-8')))
# cpu_bench = {k['Model']: k['Perf. Rating'].replace("~", "") for k in cpu_records}

cpu_bench = pd.read_csv('data/passmark/cpu.csv')
cpu_bench = {k['name']: k['passmark'] for k in cpu_bench.to_dict('records')}

def norm_name_cpu(name: str) -> str:
    return (name.lower().strip().replace("_", "-").replace("-", " ")
            .replace("intel core", "intel")
            .replace("amd ryzen", "ryzen")
            )

cpu_bench = {norm_name_cpu(k.split("@", 1)[0]): v for k, v in cpu_bench.items()}
cpu_bench['intel i7 10750h']

12012

In [74]:
# Match laptop cpu with cpu benchmark entries
def match_cpu(cpu: str, cpu_bench=cpu_bench):
    if cpu is None:
        return None
    cpu = norm_name_cpu(cpu.split("@", 1)[0])
    cpu = (cpu
           .replace("core i", "intel i")
           .replace("amd 7", "ryzen 3 ")
           .replace("processor", "")
           .replace("evo", "")
           .replace("ryzen r", "ryzen ")
           )
    while "  " in cpu:
        cpu = cpu.replace("  ", " ")
    if any(x in cpu for x in ["i3", "i5", "i7", "i9"]) and "intel" not in cpu:
        cpu = "intel " + cpu
    if cpu.startswith('mt'):
        cpu = "mediatek " + cpu
    # Strip ()
    cpu = re.sub(r'\([^)]*\)', '', cpu).strip()
    while "  " in cpu:
        cpu = cpu.replace("  ", " ")
    # Remove duplicate words
    for dup in ['i3', 'i5', 'i7', 'i9']:
        if f"{dup} {dup}" in cpu:
            cpu = cpu.replace(f"{dup} {dup}", dup)
    # Max 4 words
    cpu = " ".join(cpu.split(" ")[:4])
    for remove in ['octa', 'hexa', 'quad', 'arm', 'cortex']:
        cpu = cpu.replace(remove, "")
    cpu = cpu.strip()
    
    if cpu in cpu_bench:
        return cpu_bench[cpu]
    elif cpu + "u" in cpu_bench:
        return cpu_bench[cpu + 'u']
    else:
        print(f"CPU {cpu} not found")
        return None
    
laptops['cpu_bench'] = laptops['CPU'].apply(match_cpu)
laptops.head()

CPU  not found
CPU intel pentium silver not found
CPU ryzen 3 3720u not found
CPU intel i7 1335u not found
CPU intel i5 13th gen not found
CPU mediatek kompanio 520 not found
CPU intel ci3 1115g4 not found
CPU core not found
CPU intel pentium silver not found
CPU intel i7 12th gen not found
CPU intel i7 11th gen not found
CPU intel i5 10th gen, not found
CPU intel pentium gold 4425y not found


,id,item_id,name,brand,categories,price,specs,gpt_specs,GPU,CPU,cpu_bench
0,234878,LPASU00715,"ASUS TUF F15 Gaming Laptop 15.6"" FHD Intel i9-...",ASUS,"['Laptops, Tablet', None, None, None]",1499.0,"{'Refresh Rate': '144Hz', 'GPU': 'GeForce RTX ...","{'Brand': 'ASUS', 'Model': 'TUF F15', 'Model C...",GeForce RTX 4060,Intel i9-13900H,29687.0
1,239821,LPACE00967,"Acer Nitro 5 Gaming Laptop 15.6"" FHD Intel i9-...",ACER,"['Laptops, Tablet', None, None, None]",1299.0,"{'GPU': 'GeForce RTX 4060', 'Maximum Resolutio...","{'Brand': 'Acer', 'Model': 'Nitro 5', 'Model C...",GeForce RTX 4060,Intel i9-12900H,28524.0
2,241721,LPLEN03469,"Lenovo Legion Pro 5i Gaming Laptop 16"" QHD+ In...",LENOVO,"['Laptops, Tablet', None, None, None]",1499.0,"{'GPU': 'GeForce RTX 4060', 'Maximum Resolutio...","{'Brand': 'Lenovo', 'Model': 'Legion Pro 5i', ...",GeForce RTX 4060,Intel i7-13700HX,34110.0
3,245321,LPHEW02100,"HP 15-fd0030ca Consumer Laptop 15.6"" FHD Intel...",HP,"['Laptops, Tablet', None, None, None]",699.0,"{'GPU': 'Intel Iris Xe', 'Maximum Resolution':...","{'Brand': 'HP', 'Model': '15-fd0030ca', 'Model...",None,Intel i5-1340P,20204.0
4,245308,LPGIG00162,"GIGABYTE G5 KF Gaming Laptop 15.6"" FHD 144Hz I...",GIGABYTE,"['Laptops, Tablet', None, None, None]",1099.0,"{'GPU': 'GeForce RTX 4060', 'Maximum Resolutio...","{'Brand': 'GIGABYTE', 'Model': 'G5 KF', 'Model...",GeForce RTX 4060,Intel i7-12650H,23281.0


In [75]:
# from pathlib import Path
# import json
# 
# gpu_dicts = pd.read_csv('data/gpu.csv').to_dict('records')
# print(gpu_dicts[0])
# 
# # Exclude non-mobile GPUs from meta
# gpu_info = json.loads(Path('data/gpu_info.json').read_text())
# gpu_info = {id: v[1] for id, v in gpu_info.items()}
# gpu_cat = {id: v.get("Videocard Category") for id, v in gpu_info.items()}
# print(set(gpu_cat.values()))
# gpu_dicts = [k for k in gpu_dicts if gpu_cat[k['id']] not in {"Workstation", "Desktop"}]
# 
# # Parse gpu_bench
# gpu_bench = {k['name']: k['passmark'] for k in gpu_dicts}
# 
# # Exclude dual
# gpu_bench = {k: v for k, v in gpu_bench.items() if "dual" not in k.lower()}
# 
# def norm_name_gpu(name: str) -> str:
#     return (name.lower().strip().replace("_", "-").replace("-", " ")
#             .replace("geforce", "nvidia")
#             .replace("nvidia nvidia", "nvidia")
#             .replace("rtx", "nvidia rtx")
#             .replace("gtx", "nvidia gtx")
#             .replace("nvidia nvidia", "nvidia")
#             .replace(" laptop gpu", " (mobile)")
#             .replace(" ", " ")
#             .replace(" graphics", "")
#             )
# 
# gpu_bench = {norm_name_gpu(k): v for k, v in gpu_bench.items()}
# # Exclude non-mobile GPUs for Nvidia
# gpu_bench = {k: v for k, v in gpu_bench.items() if not k.startswith('nvidia') or 'm' in k or 'laptop' in k or '2050' in k}
# gpu_bench


In [76]:
import csv

gpu_bench = list(csv.DictReader(open('data/notebookcheck/mobile-gpu.csv', 'r', encoding='utf-8')))
gpu_bench = {k['Model']: float(k['Perf. Rating'].replace("~", "")) for k in gpu_bench if k['Perf. Rating']}
cpu_data = list(csv.DictReader(open('data/notebookcheck/mobile-cpu.csv', 'r', encoding='utf-8')))
cpu_data = {k['Model']: k for k in cpu_data}

def norm_name_gpu(name: str) -> str:
    # Remove parenthesis
    name = re.sub(r'\([^)]*\)', '', name).strip()
    return (name.lower().strip().replace("_", "-").replace("-", " ")
            .replace("geforce", "nvidia")
            .replace("nvidia nvidia", "nvidia")
            .replace("rtx", "nvidia rtx")
            .replace("gtx", "nvidia gtx")
            .replace("nvidia nvidia", "nvidia")
            .replace("radeon", "amd")
            .replace("amd rx", "amd")
            .replace("amd amd", "amd")
            .replace("ati mobility", "amd")
            .replace(" laptop gpu", " (mobile)")
            .replace(" ", " ")
            .replace(" graphics", "")
            .strip()
            )

gpu_bench = {norm_name_gpu(k): v for k, v in sorted(gpu_bench.items(), key=lambda x: x[0])}
cpu_data = {norm_name_cpu(k): v for k, v in cpu_data.items()}
gpu_bench

{'amd firepro m2000': 4.3,
 'amd firepro m3900': 4.3,
 'amd firepro m4000': 6.6,
 'amd firepro m4100': 4.0,
 'amd firepro m5100': 8.3,
 'amd firepro m5950': 6.3,
 'amd firepro m6000': 10.2,
 'amd firepro m6100': 12.0,
 'amd firepro m8900': 24.6,
 'amd firepro w4170m': 6.7,
 'amd firepro w4190m': 5.7,
 'amd firepro w5130m': 7.0,
 'amd firepro w5170m': 8.1,
 'amd firepro w7170m': 16.5,
 'amd 520': 3.8,
 'amd 530': 4.6,
 'amd 540x': 6.6,
 'amd 610m': 4.6,
 'amd 620': 3.3,
 'amd 660m': 13.6,
 'amd 680m': 18.8,
 'amd 760m': 17.2,
 'amd 780m': 20.1,
 'amd hd 6250': 1.5,
 'amd hd 6290': 0.8,
 'amd hd 6310': 3.0,
 'amd hd 6320': 1.4,
 'amd hd 6370m': 9.5,
 'amd hd 6380g': 3.1,
 'amd hd 6450m': 2.5,
 'amd hd 6470m': 9.0,
 'amd hd 6480g': 1.8,
 'amd hd 6490m': 10.9,
 'amd hd 6520g': 2.1,
 'amd hd 6530m': 6.7,
 'amd hd 6550m': 11.0,
 'amd hd 6570m': 4.8,
 'amd hd 6620g': 5.6,
 'amd hd 6630m': 4.2,
 'amd hd 6650m': 13.6,
 'amd hd 6730m': 5.0,
 'amd hd 6750m': 15.2,
 'amd hd 6770m': 16.5,
 'amd hd 

In [77]:
def match_gpu(cpu: str, gpu: str):
    cpu = match_cpu(cpu, cpu_data)
    if gpu is not None:
        gpu = gpu.lower()
    if gpu is None or any(x in gpu for x in ["iris", "uhd", "integrated", 'n/a']) or norm_name_gpu(gpu) == 'amd':
        # Check mobile integrated GPU
        if cpu is None:
            return None
        gpu = cpu['Graphics Card']
    gpu = norm_name_gpu(gpu)
    if gpu.startswith("rtx"):
        gpu = gpu.replace("rtx", "nvidia rtx")
    # If there are no space after rtx, add space
    if 'rtx' in gpu and 'rtx ' not in gpu:
        gpu = gpu.replace('rtx', 'rtx ')
    # If there are no space before "ti", add space
    if 'ti' in gpu and ' ti' not in gpu:
        gpu = gpu.replace('ti', ' ti')
    # Remove "\dgb"
    gpu = re.sub(r'\dgb', '', gpu).strip()
    if gpu.startswith("mx"):
        gpu = gpu.replace("mx", "nvidia mx")
    if gpu.startswith("iris"):
        gpu = gpu.replace("iris", "intel iris")
    if gpu.startswith("uhd"):
        gpu = gpu.replace("uhd", "intel uhd")
    if gpu.startswith("vega"):
        gpu = gpu.replace("vega", "amd vega")
    gpu = gpu.strip()
    
    if gpu.endswith("3050"):
        gpu = gpu.replace("3050", "3050 4gb")
    
    for keyword in ['', ' m', ' (mobile)', ' mobile']:
        if gpu + keyword in gpu_bench:
            return gpu_bench[gpu + keyword]
    
    print(f"GPU {gpu} not found")
    return None

laptops['gpu_bench'] = laptops.apply(lambda x: match_gpu(x['CPU'], x['GPU']), axis=1)
print(f"Found: {len(laptops['gpu_bench'].dropna())} ({len(laptops['gpu_bench'].dropna()) / len(laptops) * 100:.2f}%)")

CPU intel i5 5300u not found
GPU intel arc not found
GPU  not found
CPU intel i5 6300u not found
CPU intel pentium silver not found
CPU ryzen 3 3720u not found
GPU amd 740m not found
GPU intel arc™ not found
CPU intel i5 6300u not found
CPU intel i7 1335u not found
CPU intel i5 13th gen not found
GPU arm mali g52 2ee mc2 not found
CPU intel ci3 1115g4 not found
CPU core not found
CPU intel pentium silver not found
CPU intel i5 7300 not found
CPU intel i5 6300u not found
CPU intel i5 6300u not found
CPU intel i7 12th gen not found
CPU intel i7 6500u not found
CPU intel i5 4300u not found
CPU intel i7 11th gen not found
CPU intel i5 10th gen, not found
GPU nvidia rtx a4000 not found
CPU mediatek mt8167b not found
GPU img powervr ge8300 not found
CPU mediatek mt8167b not found
GPU img powervr ge8300 not found
Found: 577 (94.90%)


In [78]:
def calc_bench(cpu_weight: float) -> pd.DataFrame:
    # Clone laptops
    df = laptops.copy()
    
    # Normalize gpu_bench and cpu_bench to float between 0 and 1 (divide by max)
    gpu_max = max(df['gpu_bench'].dropna())
    cpu_max = max(df['cpu_bench'].dropna())
    df['gpu_bench_norm'] = df['gpu_bench'].apply(lambda x: x / gpu_max if x is not None else None)
    df['cpu_bench_norm'] = df['cpu_bench'].apply(lambda x: x / cpu_max if x is not None else None)
    
    # Drop rows with no gpu_bench or cpu_bench
    print(f"Dropping {len(df[df['gpu_bench'].isna() | df['cpu_bench'].isna()]) / len(df) * 100:.2f}%")
    df = df.dropna(subset=['gpu_bench', 'cpu_bench'])
    
    # Calculate weighted bench
    df['bench'] = df['gpu_bench_norm'] * (1 - cpu_weight) + df['cpu_bench_norm'] * cpu_weight
    
    # Drop rows with no price
    print(f"Dropping {len(df[df['price'].isna()]) / len(df) * 100:.2f}%")
    df = df.dropna(subset=['price'])
    
    # Calculate value
    df['value'] = df['bench'] / df['price'] * 1000_000
    df.sort_values('value', ascending=False).head(10)
    
    return df

calc_bench(0.3).head()

Dropping 5.43%
Dropping 0.00%


,id,item_id,name,brand,categories,price,specs,gpt_specs,GPU,CPU,cpu_bench,gpu_bench,gpu_bench_norm,cpu_bench_norm,bench,value
0,234878,LPASU00715,"ASUS TUF F15 Gaming Laptop 15.6"" FHD Intel i9-...",ASUS,"['Laptops, Tablet', None, None, None]",1499.0,"{'Refresh Rate': '144Hz', 'GPU': 'GeForce RTX ...","{'Brand': 'ASUS', 'Model': 'TUF F15', 'Model C...",GeForce RTX 4060,Intel i9-13900H,29687.0,46.3,0.637741,0.534872,0.606880,404.856791
1,239821,LPACE00967,"Acer Nitro 5 Gaming Laptop 15.6"" FHD Intel i9-...",ACER,"['Laptops, Tablet', None, None, None]",1299.0,"{'GPU': 'GeForce RTX 4060', 'Maximum Resolutio...","{'Brand': 'Acer', 'Model': 'Nitro 5', 'Model C...",GeForce RTX 4060,Intel i9-12900H,28524.0,46.3,0.637741,0.513918,0.600594,462.351180
2,241721,LPLEN03469,"Lenovo Legion Pro 5i Gaming Laptop 16"" QHD+ In...",LENOVO,"['Laptops, Tablet', None, None, None]",1499.0,"{'GPU': 'GeForce RTX 4060', 'Maximum Resolutio...","{'Brand': 'Lenovo', 'Model': 'Legion Pro 5i', ...",GeForce RTX 4060,Intel i7-13700HX,34110.0,46.3,0.637741,0.614561,0.630787,420.805300
3,245321,LPHEW02100,"HP 15-fd0030ca Consumer Laptop 15.6"" FHD Intel...",HP,"['Laptops, Tablet', None, None, None]",699.0,"{'GPU': 'Intel Iris Xe', 'Maximum Resolution':...","{'Brand': 'HP', 'Model': '15-fd0030ca', 'Model...",None,Intel i5-1340P,20204.0,10.6,0.146006,0.364016,0.211409,302.444585
4,245308,LPGIG00162,"GIGABYTE G5 KF Gaming Laptop 15.6"" FHD 144Hz I...",GIGABYTE,"['Laptops, Tablet', None, None, None]",1099.0,"{'GPU': 'GeForce RTX 4060', 'Maximum Resolutio...","{'Brand': 'GIGABYTE', 'Model': 'G5 KF', 'Model...",GeForce RTX 4060,Intel i7-12650H,23281.0,46.3,0.637741,0.419455,0.572255,520.705345


In [90]:
from pathlib import Path
from hypy_utils import write


# Export
def export(cpu_weight: float):
    laptops = calc_bench(cpu_weight)
    laptops['Model'] = laptops['gpt_specs'].apply(lambda x: x['Model'] if 'Model' in x else None)
    laptops['RAM'] = laptops['gpt_specs'].apply(lambda x: int(x['RAM'].split("GB", 1)[0].strip()) if 'RAM' in x else None)
    laptops['Storage'] = laptops['gpt_specs'].apply(lambda x: x['Storage'].replace("SSD", "").replace("NVMe", "").strip() if 'Storage' in x else None)
    laptops['GPU'] = laptops['GPU'].apply(lambda x: x.replace("Graphics", "") if x is not None else None)
    df = laptops[['item_id', 'Model', 'CPU', 'GPU', 'RAM', 'Storage', 'price', 'bench', 'cpu_bench', 'gpu_bench', 'value']]
    df = df.sort_values('value', ascending=False)
    df['bench'] = df['bench'] * 100
    Path(f'data/computed').mkdir(exist_ok=True, parents=True)
    df.to_csv(f'data/computed/canada_computers_laptops_bench_{cpu_weight}.csv', index=False)

    html_template = '''<!DOCTYPE html>
    <html>
        <head>
        <style>
            tbody tr:nth-child(even) {background-color: lightblue;}
            body {font-family: Arial, Helvetica, sans-serif;}
        </style>
        </head>
        <body>
            {{table}}
        </body>
    </html>
    '''
    
    # Color bars for price, bench, value, cpu_bench, gpu_bench
    df = (df.style.bar(subset=['price'], color='#d65f5f', vmax=1500)
            .bar(subset=['bench'], color='#5fba7d')
            .bar(subset=['value'], color='#5fba7d')
            .bar(subset=['cpu_bench'], color='#5fba7d')
            .bar(subset=['gpu_bench'], color='#5fba7d')
            .bar(subset=['RAM'], color='#5fba7d', vmax=32)
    )
    
    
    # Format price
    df = df.format({'price': '$ {:,.0f}', 'bench': '{:.0f}%', 'cpu_bench': '{:,.0f}', 'gpu_bench': '{:.1f}', 'value': '{:,.0f}', 
                    'RAM': '{:,.0f}'})
    
    # Justify numbers to right, non-numbers to left
    df = df.set_properties(**{'text-align': 'right'}, subset=pd.IndexSlice[:, ['price', 'bench', 'cpu_bench', 'gpu_bench', 'value']])
    
    # df = df.set_table_styles([{"selector":"tbody tr:nth-child(even)","props":[("background-color","lightgrey")]}])
    
    write(f'data/computed/canada_computers_laptops_bench_{cpu_weight}.html', html_template.replace('{{table}}', df.to_html()))
    

for i in [0.1, 0.3, 0.5, 0.7, 0.9]:
    export(i)

Dropping 5.43%
Dropping 0.00%
Dropping 5.43%
Dropping 0.00%
Dropping 5.43%
Dropping 0.00%
Dropping 5.43%
Dropping 0.00%
Dropping 5.43%
Dropping 0.00%
